In [1]:
%load_ext autoreload
%autoreload 2
%pwd

'/home/lightpeak008/qtmpy/integration'

In [2]:
from qtmpy import qtmconfig
from qtmpy.gspace.fft.backend import get_FFTBackend

In [3]:
print(qtmconfig.fft_available_backends)
print(qtmconfig.fft_backend)
print(qtmconfig.fft_threads)
print(qtmconfig.pyfftw_planner, qtmconfig.pyfftw_flags)
qtmconfig.use_gpu = True

['mkl_fft', 'pyfftw', 'scipy', 'numpy']
mkl_fft
1
FFTW_MEASURE ('FFTW_DESTROY_INPUT',)


In [4]:
import numpy as np

shape = (18, 18, 18)
arr_r = np.random.rand(2, *shape)
arr_r = arr_r[0] + 1j * arr_r[1]

arr_g = np.fft.fftn(arr_r)


for backend in qtmconfig.fft_available_backends:
    FFTBackend = get_FFTBackend(backend)
    print(FFTBackend)
    
    flag = True
    # 3D FFT Test
    out = FFTBackend.create_buffer(shape)
    fft = FFTBackend(out, (0, 1, 2))
    out[:] = arr_r
    fft.fft()
    if not np.allclose(out, arr_g):
        flag = False
        print('forward fail')
    out[:] = arr_g
    fft.ifft(True)
    if not np.allclose(out, arr_r):
        flag = False
        print('backward with norm failed')
    out[:] = arr_g
    fft.ifft(False)
    normfac = np.prod(shape)
    if not np.allclose(out, normfac*arr_r):
        flag = False
        print('backward unnormed failed')
    
    print('Going to 2D now')
    out = FFTBackend.create_buffer(shape)
    axes = (1, 2)
    fft = FFTBackend(out, axes)
    out[:] = arr_r
    fft.fft()
    if not np.allclose(out, np.fft.fftn(arr_r, axes=axes)):
        flag = False
        print('forward fail')
    out[:] = arr_g
    fft.ifft(True)
    if not np.allclose(out, np.fft.ifftn(arr_g, axes=axes)):
        flag = False
        print('backward with norm failed')
    out[:] = arr_g
    fft.ifft(False)
    normfac = shape[0] * shape[2]
    if not np.allclose(
        out, normfac * np.fft.ifftn(arr_g, axes=axes)):
        flag = False
        print('backward unnormed failed')
    
    print("Pass" if flag else "Fail")

<class 'qtmpy.gspace.fft.backend.mklfft_.MKLFFTWrapper'>
Going to 2D now
Pass
<class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
Going to 2D now
Pass
<class 'qtmpy.gspace.fft.backend.scipy_.SciPyFFTWrapper'>
Going to 2D now
Pass
<class 'qtmpy.gspace.fft.backend.numpy_.NumPyFFTWrapper'>
Going to 2D now
Pass


In [5]:
if qtmconfig.use_gpu:
    import cupy as cp
    arr_r = cp.asarray(arr_r)
    arr_g = cp.fft.fftn(arr_r)
    
    FFTBackend = get_FFTBackend('cupy')
    print(FFTBackend)
    
    flag = True
    # 3D FFT Test
    out = FFTBackend.create_buffer(shape)
    fft = FFTBackend(out, (0, 1, 2))
    out[:] = arr_r
    fft.fft()
    if not np.allclose(out, arr_g):
        flag = False
        print('forward fail')
    out[:] = arr_g
    fft.ifft(True)
    if not np.allclose(out, arr_r):
        flag = False
        print('backward with norm failed')
    out[:] = arr_g
    fft.ifft(False)
    normfac = np.prod(shape)
    if not np.allclose(out, normfac*arr_r):
        flag = False
        print('backward unnormed failed')
    
    print('Going to 2D now: ', flag)
    out = FFTBackend.create_buffer(shape)
    axes = (1, 2)
    fft = FFTBackend(out, axes)
    out[:] = arr_r
    fft.fft()
    if not np.allclose(out, np.fft.fftn(arr_r, axes=axes)):
        flag = False
        print('forward fail')
    out[:] = arr_g
    fft.ifft(True)
    if not np.allclose(out, np.fft.ifftn(arr_g, axes=axes)):
        flag = False
        print('backward with norm failed')
    out[:] = arr_g
    fft.ifft(False)
    normfac = shape[0] * shape[2]
    if not np.allclose(
        out, normfac * np.fft.ifftn(arr_g, axes=axes)):
        flag = False
        print('backward unnormed failed')
    
    print("Pass" if flag else "Fail")

<class 'qtmpy.gspace.fft.backend.cupy_.CuPyFFTWrapper'>
Going to 2D now:  True
Pass


In [6]:
shape = (45, 45, 45)
numg = 5000
normalise_idft = False

arr_g = np.random.rand(2, numg)
arr_g = arr_g[0] + 1j * arr_g[1]

idxg = np.random.choice(np.prod(shape) - 1, numg, replace=False) + 1
idxg[0] = 0

inp = np.zeros(shape, dtype='c16')
inp.put(idxg, arr_g)
arr_r = np.fft.ifftn(inp)

normfac = np.prod(shape)
if normalise_idft:
    arr_r_norm = arr_r
else:
    arr_r_norm = arr_r * normfac

In [7]:
from qtmpy.gspace.fft import FFT3DFull
from qtmpy.gspace.fft import FFT3DSticks


fft = FFT3DFull(shape, idxg, normalise_idft)
out = fft.create_buffer(shape)
fft.g2r(arr_g, out)
print(np.allclose(out, arr_r_norm))
out = fft.create_buffer(numg)
fft.r2g(arr_r, out)
print(np.allclose(out, arr_g))

fft = FFT3DSticks(shape, idxg, normalise_idft)
out = fft.create_buffer(shape)
fft.g2r(arr_g, out)
print(np.allclose(out, arr_r_norm))
out = fft.create_buffer(numg)
fft.r2g(arr_r, out)
print(np.allclose(out, arr_g))

True
True
True
True


In [8]:
%load_ext line_profiler

In [10]:
from qtmpy.gspace.fft.full import FFT3DFull
# from qtmpy.gspace.fft.sticks import FFT3DSticksv1, FFT3DSticksv2

qtmconfig.fft_backend = 'pyfftw'
print(qtmconfig.fft_backend)


fft = FFT3DFull(shape, idxg, normalise_idft)
print(fft.FFTBackend)
out = fft.create_buffer(shape)
%timeit fft.g2r(arr_g, out)
print(np.allclose(out, arr_r_norm))

# fft = FFT3DSticksv1(shape, idxg, normalise_idft)
# out = fft.create_buffer(shape)
# %timeit fft.g2r(arr_g, out)
# print(np.allclose(out, arr_r_norm))

# fft = FFT3DSticksv2(shape, idxg, normalise_idft)
# out = fft.create_buffer(shape)
# %timeit fft.g2r(arr_g, out)
# print(np.allclose(out, arr_r_norm))

fft = FFT3DFull(shape, idxg, normalise_idft)
out = fft.create_buffer(shape)
%timeit fft.g2r(arr_g, out)
print(np.allclose(out, arr_r_norm))

pyfftw
<class 'qtmpy.gspace.fft.backend.pyfftw_.PyFFTWFFTWrapper'>
2.04 ms ± 65.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
True
2.2 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
True


In [ ]:
print(qtmconfig.fft_backend)

In [ ]:
from qtmpy.gspace.fft.sticks import FFT3DSticksv1

fft = FFT3DSticksv1(shape, idxg, True)
print(len(fft.fftz_map[0]), len(fft.ffty_map))

out = fft.create_buffer(shape)
%timeit fft.g2r(arr_g, out)
print(np.allclose(out, arr_r))
out = fft.create_buffer(numg)
# %lprun -f fft.r2g fft.r2g(arr_r, out)
print(np.allclose(out, arr_g))

In [24]:
from qtmpy.gspace.fft.sticks import FFT3DSticksv2

fft = FFT3DSticksv2(shape, idxg, True)
print(fft.numsticks, fft.numplanes)


out = fft.create_buffer(shape)
%timeit fft.g2r(arr_g, out)
print(np.allclose(out, arr_r))
out = fft.create_buffer(numg)
# %lprun -f fft.r2g fft.r2g(arr_r, out)
print(np.allclose(out, arr_g))

1857 45
78.9 ms ± 675 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
True
False


In [9]:
if qtmconfig.use_gpu:
    idxg = cp.asarray(idxg)
    arr_g = cp.asarray(arr_g)
    arr_r = cp.asarray(arr_r)
    
    fft = FFT3DFull(shape, idxg, True, 'cupy')
    out = fft.create_buffer(shape)
    fft.g2r(arr_g, out)
    print(np.allclose(out, arr_r))
    out = fft.create_buffer(numg)
    fft.r2g(arr_r, out)
    print(np.allclose(out, arr_g))

    fft = FFT3DSticks(shape, idxg, True, 'cupy')
    out = fft.create_buffer(shape)
    fft.g2r(arr_g, out)
    print(np.allclose(out, arr_r))
    print(out / arr_r)
    out = fft.create_buffer(numg)
    fft.r2g(arr_r, out)
    print(np.allclose(out, arr_g))

True
True
False
[[[  0.64444466+2.48430408e-01j  -3.22284852-3.88378457e+01j
    -0.        +0.00000000e+00j ...  -0.        +0.00000000e+00j
   -12.9061091 +1.20842692e+00j   0.        +0.00000000e+00j]
  [  0.        +0.00000000e+00j  -0.        +0.00000000e+00j
     0.        -0.00000000e+00j ... -25.51570315+1.98074408e+01j
    -9.92266186-3.39324147e+01j -23.97441252+1.79034460e+02j]
  [  8.974545  -4.38646289e+01j   0.        +0.00000000e+00j
   -26.7031118 +4.47513628e+00j ...   0.        -0.00000000e+00j
    -7.3948697 +1.22879036e+01j   0.        -0.00000000e+00j]
  ...
  [-54.00920434+3.96967191e+01j   0.        -0.00000000e+00j
   -52.104017  +4.96639233e+01j ...   0.        +0.00000000e+00j
    10.50941092-2.60032751e+01j   0.        +0.00000000e+00j]
  [ 42.04897988-8.31289234e+00j   0.        +0.00000000e+00j
     0.        -0.00000000e+00j ...  -0.6899274 +8.93310768e+01j
   -37.52330286+8.06723630e+00j   0.        +0.00000000e+00j]
  [ 28.62567061-1.36082782e+01j   0.  

In [40]:
import cupy as cp
from cupyx.scipy.fft import get_fft_plan, fftn, ifftn

a = cp.random.rand(2, 100, 100)
a = a[0] + 1j * a[1]
print(a.shape, a.dtype, a.flags)

fft_plan = get_fft_plan(
    cp.empty((a.shape[0], ), dtype='c16'),
    axes=(0, ), value_type='C2C'
)

(100, 100) complex128   C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True


In [42]:
b = a.copy()
for i in range(100):
    fftn(b[i], axes=(0, ), plan=fft_plan, overwrite_x=True)
c = cp.fft.fftn(a.copy(), axes=(1, ))
cp.allclose(b, c)

array(False)

In [30]:
print(b/c)

[[ 0.01151109-0.00442619j -0.25290139-0.04505716j  0.0062705 -0.04562721j
  ... -0.11520014-0.10994552j -0.01421395+0.12010252j
  -0.05416686+0.05385627j]
 [ 0.00334864-0.00259917j -0.03544413+0.13930451j  1.88561429-0.62434709j
  ...  0.09142189-0.27307002j -0.06467797+0.29195827j
   0.14922066+0.22925666j]
 [ 0.00763572+0.002415j    0.11760314-0.06326843j -0.19983771+0.28597952j
  ...  0.17403785+0.17290627j  0.26428007-0.15900969j
   0.04590734-0.1934446j ]
 ...
 [ 0.00962284+0.00850002j  0.5336527 -0.0310926j   0.1822149 +0.22511929j
  ... -0.25483874-0.058187j   -0.00651628-1.33787736j
  -0.7103268 +0.94186887j]
 [ 0.00698721+0.0028195j  -0.31296099+0.39665832j  0.18564964+0.14748146j
  ...  0.00357752-0.13085192j -0.14140393+0.09803637j
  -0.10200116+0.01657402j]
 [ 0.00410998+0.00263672j  0.19523014-0.05560594j  0.17796905-0.3080466j
  ...  0.05857186+0.23944494j -0.17306527-0.00386679j
  -0.01804388+0.40914555j]]


In [ ]:
cp.cuda.cufft.